# Space Titanic Competition

Welcome to the year 2912, where your data science skills are needed to solve a cosmic mystery. We've received a transmission from four lightyears away and things aren't looking good.

The Spaceship Titanic was an interstellar passenger liner launched a month ago. With almost 13,000 passengers on board, the vessel set out on its maiden voyage transporting emigrants from our solar system to three newly habitable exoplanets orbiting nearby stars.

While rounding Alpha Centauri en route to its first destination—the torrid 55 Cancri E—the unwary Spaceship Titanic collided with a spacetime anomaly hidden within a dust cloud. Sadly, it met a similar fate as its namesake from 1000 years before. Though the ship stayed intact, almost half of the passengers were transported to an alternate dimension!

To help rescue crews and retrieve the lost passengers, you are challenged to predict which passengers were transported by the anomaly using records recovered from the spaceship’s damaged computer system.

Help save them and change history!

<img src="images/competition_image.jpeg" width="500">

[link to competition homepage](https://www.kaggle.com/c/spaceship-titanic/overview)

### Evaluation Criteria

Submissions are evaluated based on their classification accuracy, the percentage of predicted labels that are correct.

Accuracy = (TP + TN) / (TP + TN + FP + FN)

Where TP = True Positives, TN = True Negatives, FP = False Positives, and FN = False Negatives.

### Submission Format

The submission format for the competition is a csv file with the following format:

>PassengerId,Transported<br>
>0013_01,False<br>
>0018_01,False<br>
>0019_01,False<br>
>0021_01,False<br>
>etc.

In this competition your task is to predict whether a passenger was transported to an alternate dimension during the Spaceship Titanic's collision with the spacetime anomaly. To help you make these predictions, you're given a set of personal records recovered from the ship's damaged computer system.

### File and Data Field Descriptions
* train.csv - Personal records for about two-thirds (~8700) of the passengers, to be used as training data.
    * PassengerId - A unique Id for each passenger. Each Id takes the form gggg_pp where gggg indicates a group the passenger is travelling with and pp is their number within the group. People in a group are often family members, but not always.
    * HomePlanet - The planet the passenger departed from, typically their planet of permanent residence.
    * CryoSleep - Indicates whether the passenger elected to be put into suspended animation for the duration of the voyage. Passengers in cryosleep are confined to their cabins.
    * Cabin - The cabin number where the passenger is staying. Takes the form deck/num/side, where side can be either P for Port or S for Starboard.
    * Destination - The planet the passenger will be debarking to.
    * Age - The age of the passenger.
    * VIP - Whether the passenger has paid for special VIP service during the voyage.
    * RoomService, FoodCourt, ShoppingMall, Spa, VRDeck - Amount the passenger has billed at each of the Spaceship Titanic's many luxury amenities.
    * Name - The first and last names of the passenger.
    * Transported - Whether the passenger was transported to another dimension. This is the target, the column you are trying to predict.
* test.csv - Personal records for the remaining one-third (~4300) of the passengers, to be used as test data. Your task is to predict the value of Transported for the passengers in this set.
* sample_submission.csv - A submission file in the correct format.
    * PassengerId - Id for each passenger in the test set.
    * Transported - The target. For each passenger, predict either True or False.

### Questions before we start

* What do you think are the "easiest" features to start with?
* What engineered features can you think of?
    * What will we need to do to create these features?
* What do you *think* will be the most influential factor?
* Some of these features are numeric, others are non-numeric categorical features. What type of pre-processing do you need to do to make these features readable for our model fitting?

In [ ]:
import numpy as np
import pandas as pd
import os

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn import model_selection
from sklearn import metrics
from sklearn import model_selection
from sklearn import ensemble
from sklearn import pipeline

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

### Load the data

**Instructions:**
The first thing you need to do is load your data. syntax for pandas read_csv is below. You will need to do this for both testing and training data (two separate dataframes). I'll be using `df_train` and `df_test`.

`df = pd.read_csv(file)`
    
**hints**: 

Testing data are in `./data/test.csv`

Training data are in `./data/train.csv`

In [ ]:
# Write your code here

### Fill NaN values

**Instructions:** There are multiple ways to do this. For now, let's use the *mode* of each column to fill in the NaN values.

**hints:**

This command will show you how many NaNs are in each column of a dataframe: `df.isna().sum()`

To fill the NaN values for a column named `col` inplace with the mode, use: `df[col].fillna(df[col].mode()[0], inplace=True)`

To get a list of columns, use: `df.columns.tolist()`

Remember to do this for both training and testing datasets.

In [ ]:
# Write your code here

### Inspect the data

**Instructions:** It's important to understand the data you are working with. Take this time to look at your data in the dataframe and using visualization. You can use either matplotlib or seaborn, but I'll give a few useful hints below.

Start by looking at these features: 'HomePlanet', 'Destination', 'CryoSleep', 'VIP', 'Age'

**hints:**

To show the first five rows of a dataframe: `df.head(5)`

To show the value counts in a column `col`: `df[col].value_counts()`

To plot the value counts as a bar plot for a column `col`: `df[col].value_counts().plot.bar()`

To plot a histogram of a column `col`: `df[col].plot.hist()`

In [ ]:
# Write your code here

**Edit this cell with your observations**

Notes: Write down your observations about each of these features:
* Home Planet
* Destination
* Cryo Sleep
* VIP

**Edit this cell with your observations**

Notes: Write down your observations about age distribution.

### Function definition cells (edit these at your own risk)

In [ ]:
def preprocessing_x(df_train, df_test, y, features, encoded_features):
    """ 
    Pre-process the data to prepare it for model fitting.
    
    Parameters:
        df_train: Training dataframe
        df_test: Testing dataframe
        y: Targets
        features: List of features to use
        encoded features: List of features that need to be encoded
    
    Returns:
        X_train: Training features
        X_val: Validation features
        X_test: Testing features
        y_train: Training targets
        y_val: Validation targets
    """
    
    # define X values as a subset of the dataframe
    X = df_train[features]
    X_test = df_test[features]
    
    # Use label encoder to make non-numeric features integers
    for feature in encoded_features:
        le_x = preprocessing.LabelEncoder()
        le_x.fit(X[feature])
        X.loc[:,feature] = le_x.transform(X[feature])
        X_test.loc[:,feature] = le_x.transform(X_test[feature])

    # Split into training and validation data
    X_train, X_val, y_train, y_val = model_selection.train_test_split(X, y, test_size=0.33)
    
    return X_train, X_val, X_test, y_train, y_val

In [ ]:
def make_classification(clf, features, X_train, y_train, X_val, y_val, X_test):
    """
    Given a classifier and a set of features, fit the model.
    
    Parameters:
        clf: Classifier from sklearn
        features: List of features to use
        X_train: Training features
        y_train: Training targets
        X_val: Validation features
        y_val: Validation targets
        X_test: Testing features
        
    Returns:
        y_test_pred: Predicted test targets
    """
    
    # Fit to training data and predict for validation data
    pipe = pipeline.make_pipeline(preprocessing.StandardScaler(), clf)
    pipe.fit(X_train, y_train)
    y_val_pred = pipe.predict(X_val)
    
    # Print the accuracy score
    print(f'Accuracy: {pipe.score(X_val, y_val)}')
    
    # Make the confusion matrix and display it
    cm = metrics.confusion_matrix(y_val, y_val_pred)
    disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cm, 
                                          display_labels=pipe.classes_)
    disp.plot()
    plt.show()
    
    # Calculate feature importances, print them, and display them
    feature_importance = pipe.steps[1][1].feature_importances_.tolist()

    print('Feature importance:')
    for i in np.arange(len(feature_importance)):
        print(features[i], '\t', feature_importance[i])
    
    plt.bar(features, feature_importance)
    plt.title('Feature Importance')
    plt.xticks(rotation = 90);
    plt.show()
    
    # Try it on the test data for submission
    y_test_pred = pipe.predict(X_test)
    
    return y_test_pred

## Phase 1: Simple model with limited features

**Instructions:**
For our initial model, we'll define a set of simple features. Let's start with 'HomePlanet', 'Destination', 'CryoSleep', 'VIP', and 'Age'. Note that all of these except Age are non-numeric categorical features that need to be encoded. The target data also need to be converted to integer values (0, 1) rather than boolean values (False, True).

**hints:**
For the simple case, I'll give you the syntax. For the complex case, you'll need to do more typing.

In [ ]:
# List of features
simple_features = ['HomePlanet', 'Destination', 'CryoSleep', 'VIP', 'Age']

# List of non-numeric features
encoded_simple_features = ['HomePlanet', 'Destination', 'CryoSleep', 'VIP']

# Target column name
target_col = 'Transported'

In [ ]:
# Extract the target column
df_targets = df_train[target_col]

# Convert target to integer type
y = df_targets.astype(int)

In [ ]:
# Run pre-processing for the simple features list
X_train, X_val, X_test, y_train, y_val = preprocessing_x(df_train, 
                                                        df_test, y, simple_features, 
                                                        encoded_simple_features)

In [ ]:
# Use the random forest classifier 
clf = ensemble.RandomForestClassifier()

# Run the classification
y_test_pred = make_classification(clf, simple_features, X_train, 
                                  y_train, X_val, y_val, X_test)

**Edit this cell with your observations**

Notes: Write down your observations below.
* How are we doing so far? Is this a 'good' score?
* Which are the most important features? The least important?
    * Does this surprise you?

In [ ]:
## This cell shows how to save the data to csv, but let's not mess with that right now
# y_simple_submission = pd.DataFrame({'PassengerId': df_test.PassengerId, 
#                                     'Transported': y_test_pred.astype(bool)})
# y_simple_submission.to_csv('./data/simple_submission.csv', index=False)

## Phase 2: Complex model with added features

**Instructions:**
For our second model, we'll define a set of expanded features.

* It's worth noting that `Cabin` actually contains three values: deck/num/side
    * Probably deck and side are more significant than num
* Perhaps the total amount spent across 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', and 'VRDeck' would be an interesting feature to include?
* If someone is traveling with a group, could it be possible that they either transport or not together?
    * Let's assume that the group is important, but the number in the group isn't

**hints:**

If you have a column that contains text that you want to split on the `/` character, the following may be helpful: `df[['col1', 'col2', 'col3']] = df[col].str.split(pat="/", expand=True)`

Similarly, if you want to split on `_`, you could use: `df[['col1', 'col2']] = df[col].str.split(pat="_", expand=True)`

In [ ]:
# Write your code here

**Edit this cell with your observations**

Notes: Write down your observations below.
* How are we doing so far? Is this a 'better' score?
* Which are the most important features? The least important?
    * Are there any new features that are praticularly useful?

### Random observation time:

Something the suprised me was that the VIP status didn't matter more, but VIPs were less likely to be transported than non-VIPs.

In [ ]:
df_train[df_train['VIP']==True]['Transported'].value_counts().plot.bar()

In [ ]:
df_train[df_train['VIP']==False]['Transported'].value_counts().plot.bar()

## Phase 3: Optimize the hyper-parameters

Okay, so we have a decent model. But notice that we didn't do ANY hyper-parameter tuning so far. We just went with the default values.

For more about grid search, click [here](https://scikit-learn.org/stable/modules/grid_search.html)

We will be using a randomized search. See [here](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV) for more details.


**I'm giving these cells to you "for free" but you can feel free to mess around with the values and such if you'd like.**

Note: credit for the following few cells goes to this article: https://towardsdatascience.com/hyperparameter-tuning-the-random-forest-in-python-using-scikit-learn-28d2aa77dd74

In [ ]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [ ]:
# Set up our randomized grid search
clf_random = model_selection.RandomizedSearchCV(estimator = clf, 
                               param_distributions = random_grid, 
                               n_iter = 30, cv = 3, verbose=2, 
                               random_state=42, n_jobs = -1)

In [ ]:
# Find the best fit
clf_random.fit(X_train, y_train)

In [ ]:
# Print the best parameters
clf_random.best_params_

In [ ]:
# Run the classification model with the best estimator
best_random = clf_random.best_estimator_
y_test_pred = make_classification(best_random, complex_features, 
                                  X_train, y_train, X_val, y_val, X_test)

In [ ]:
## Data writing cell
# y_best_submission = pd.DataFrame({'PassengerId': df_test.PassengerId, 
#                                     'Transported': y_test_pred.astype(bool)})
# y_best_submission.to_csv('./data/best_submission.csv', index=False)

**Edit this cell with your observations**

Notes: Write down your observations below.
* How did we do? Is this the 'best' score?
* Did tuning improve the score?
* Would tuning matter more or less in other models?
* Take a look at the [leaderboard](https://www.kaggle.com/c/spaceship-titanic/leaderboard#)

## Phase 4: Freeform time

Want to play aorund with some other models? Retune parameters? Add more features? Here's your chance to try to improve. Work alone or ask you neighbor for help. Maybe form a team on kaggle and create your own submission!